# Classification Model of Purchase Intentions
### DSCI100 Project Proposal Group 13

In [9]:
library(tidyverse)
library(tidymodels)

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3      ✔ yardstick 0.0.7 

Warning message:
“package ‘broom’ was built under R version 4.0.2”
Warning message:
“package ‘dials’ was built under R version 4.0.2”
Warning message:
“package ‘infer’ was built under R version 4.0.3”
Warning message:
“package ‘modeldata’ was built under R version 4.0.1”
Warning message:
“package ‘parsnip’ was built under R version 4.0.2”
Warning message:
“package ‘recipes’ was built under R version 4.0.1”
Warning message:
“package ‘tune’ was built under R version 4.0.2”
Warning message:
“package ‘workflows’ was built under R version 4.0.2”
Warning message:
“package ‘yardstick’ was built under R version 4.0.2”
── Conflicts ────

In [11]:
shoppers <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00468/online_shoppers_intention.csv") %>%
    select(BounceRates:Month, Weekend:Revenue) %>%
    mutate(Revenue = as_factor(Revenue))

shoppers_split <- initial_split(shoppers, prop = 0.5, strata = Revenue)
shoppers_train <- training(shoppers_split)
shoppers_test <- testing(shoppers_split)

head(shoppers_train)

Parsed with column specification:
cols(
  Administrative = col_double(),
  Administrative_Duration = col_double(),
  Informational = col_double(),
  Informational_Duration = col_double(),
  ProductRelated = col_double(),
  ProductRelated_Duration = col_double(),
  BounceRates = col_double(),
  ExitRates = col_double(),
  PageValues = col_double(),
  SpecialDay = col_double(),
  Month = col_character(),
  OperatingSystems = col_double(),
  Browser = col_double(),
  Region = col_double(),
  TrafficType = col_double(),
  VisitorType = col_character(),
  Weekend = col_logical(),
  Revenue = col_logical()
)



BounceRates,ExitRates,PageValues,SpecialDay,Month,Weekend,Revenue
<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<fct>
0.20000000,0.20000000,0,0.0,Feb,FALSE,FALSE
0.05000000,0.14000000,0,0.0,Feb,FALSE,FALSE
0.01578947,0.02456140,0,0.0,Feb,FALSE,FALSE
0.00000000,0.10000000,0,0.8,Feb,FALSE,FALSE
0.00000000,0.02222222,0,0.4,Feb,FALSE,FALSE
0.00000000,0.02857143,0,0.0,Feb,FALSE,FALSE
